In [ ]:
import numpy
CC = ComplexField(32)
pi = CC.pi()
I = CC.gen()

def generate_random_hnp_instance(leak_size, number_of_signatures, prime, key):
    HNP_instance = []
    bound = (prime-1) / 2**(leak_size+1)

    for j in range(number_of_signatures):
        c = randrange(1, prime)
        k = randrange(-round(bound), floor(bound))
        h = (k - c*key) % prime
        HNP_instance.append((c, h))
    return HNP_instance

def sort_and_difference(hnp_instance, number_of_iterations, prime):
    new_instance = hnp_instance
    for i in range(number_of_iterations):
        new_instance.sort(key=lambda x:x[0])
        helper_list = []
        for j in range(len(new_instance)-1):
            helper_list.append(((new_instance[j+1][0]-new_instance[j][0]), (new_instance[j+1][1]-new_instance[j][1])% prime))
        new_instance = helper_list.copy()
    return new_instance

def bleichenbacher(hnp_instance, prime):
    n = max(hnp_instance, key=lambda x: x[0])[0]
    print(n)
    z = [0] * (n+1)
    for i in range(len(hnp_instance)-1):
        z[hnp_instance[i][0]] += (1/len(hnp_instance)) * exp(2*pi*I* hnp_instance[i][1]/prime)
    frequencies = numpy.fft.ifft(z)
    magnitudes = numpy.abs(frequencies)
    
    return numpy.argmax(magnitudes)*prime/(2*n)

def transform_hnp_instance_to_lattice_reduction_form(hnp_instance, prime):
    for i in range(len(hnp_instance)):
        hnp_instance[i][1] = -1 * hnp_instance[i][1] % prime 


In [ ]:

LEAK_SIZE = 4
NUMBER_OF_SIGNATURES = 100000

prime = 1012115836375
key = 7497057881

hnp_instance = generate_random_hnp_instance(LEAK_SIZE, NUMBER_OF_SIGNATURES, prime, key)

#hnp_instance = sort_and_difference(hnp_instance, 5, prime)

#print(bleichenbacher(hnp_instance, prime))

#timhle se to mozna dojebe :D
transform_hnp_instance_to_lattice_reduction_form(hnp_instance, prime)

#find reduced basis and target vector
basis = matrix(len(hnp_instance) + 1, sparse=True)
basis[len(hnp_instance) + 1, len(hnp_instance) + 1] = 1/prime
for i in range(len(hnp_instance)):
    basis[i,i] = prime
    basis[len(hnp_instance)+1, i] = hnp_instance[i][0]

reduced_basis = basis.LLL()
target = [0] * (len(hnp_instance) + 1)

for i in range(len(hnp_instance)):
    target[i] = hnp_instance[i][1]

#apply the nearest plane procedure to the target vector

#switch to column representation
reduced_basis = reduced_basis.transpose()
gramm_schmidt_basis, coefficients = reduced_basis.QR()

#nearest plane
b = target.copy()
for i in range(len(hnp_instance)):
    b = b - reduced_basis.column(i) * gramm_schmidt_basis.column(i)


print("if this isnt 1 then divide the above term", gramm_schmidt_basis.column(1) * gramm_schmidt_basis.column(1))

print(target - b)
print("the private key is:", (target-b)[len(hnp_instance)])






/tmp/ipykernel_7384/3796661791.py:12: DeprecationWarning: the default rounding for rationals, currently `away`, will be changed to `even`.
See https://github.com/sagemath/sage/issues/35473 for details.
  k = randrange(-round(bound), floor(bound))


362614
21441736543.632484
